In [16]:
import functools
import operator
import requests
import os
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent
from langchain_core.messages import HumanMessage, BaseMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI

from typing import TypedDict, Annotated, Sequence

from langchain_community.tools.tavily_search import TavilySearchResults

import gradio as gr
from decouple import config
from langchain_google_genai import ChatGoogleGenerativeAI

In [17]:
llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
)

In [18]:
@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Used to process content found on the internet."""
    response = requests.get(url=url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text()

In [19]:
@tool("internet_search_tool", return_direct=False)
def internet_search_tool(query: str) -> str:
    """Search provided query on the internet using DuckDuckGo"""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return results if results else "No results found"

In [20]:
# tools = [TavilySearchResults(max_results=1), process_search_tool]
tools = [internet_search_tool, process_search_tool]

In [21]:
def create_agents(llm: ChatGoogleGenerativeAI,
                  tools: list,
                  system_prompt: str) -> AgentExecutor:
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
    agent = create_react_agent(llm, tools, prompt)
   
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [22]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [23]:
members = ["news_correspondent", "news_editor", "ads_writter"]

In [24]:
system_prompt = (
    "As a supervisor, your role is to oversee the insight between these"
    " workers: {members}. Based on the user's request,"
    " determine which worker should take the next action. Each worker is responsible for"
    " executing a specific task and reporting back thier findings and progress."
    " Once all tasks are completed, indicate 'FINISH'."
)

options = ["FINISH"] + members

In [25]:
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {"next": {"title": "Next", "anyOf": [{"enum": options}]}},
        "required": ["next"]
    }
}

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
    ("system",
     "Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}"),
]).partial(options=str(options), members=", ".join(members))


In [27]:
supervisor_chain = (prompt | llm.bind_functions(
    functions=[function_def], function_call="route") | JsonOutputFunctionsParser())

In [28]:
news_correspondent_agent = create_agents(
    llm,
    tools,
    """Answer the following questions as best you can. You have access to the following tools:

            {tools}{tool_names}
    Your primary role is to function as an intelligent news research assistant, adept at scouring 
    the internet for the latest and most relevant trending stories across various sectors like politics, technology, 
    health, culture, and global events. You possess the capability to access a wide range of online news sources, 
    blogs, and social media platforms to gather real-time information."""
)

# print(news_correspondent_agent)

In [29]:
news_correspondent_node = functools.partial(
    agent_node, agent=news_correspondent_agent, name="news_correspondent"
)

In [30]:
news_editor_agent = create_agents(
    llm, tools,
    """Answer the following questions as best you can. You have access to the following tools:

            {tools}{tool_names}
    You are a news editor. Do step by step approach. 
        Based on the provided content first identify the list of topics,
        then search internet for each topic one by one
        and finally find insights for each topic one by one that can aid you 
        in writting a useful news edition for AI-nes corp.
        Include the insights and sources in the final response
        """)

In [31]:
news_editor_node = functools.partial(
    agent_node, agent=news_editor_agent, name="news_editor")


In [32]:
ads_writter_agent = create_agents(
    llm, tools,
    """Answer the following questions as best you can. You have access to the following tools:

            {tools}{tool_names}
    You are an ads writter for AI-news corp. Given the publication generated by the
    news editor, your work if to write ads that relate to that content. Use the internet 
    to search for content to write ads based off on. Here is a description of your task:
    
    To craft compelling and relevant advertisements for 'AI News' publication, complementing the content written by the news editor.
    Contextual Ad Placement: Analyze the final report content from the news editor in-depth to identify key themes, topics, 
    and reader interests. Place ads that are contextually relevant to these findings, thereby increasing potential customer engagement.
    Advanced Image Sourcing and Curation: Employ sophisticated web search algorithms to source high-quality, relevant images for each ad. 
    Ensure these images complement the ad content and are aligned with the publication's aesthetic standards.
    Ad-Content Synchronization: Seamlessly integrate advertisements with the report, ensuring they enhance rather than disrupt the reader's 
    experience. Ads should feel like a natural extension of the report, offering value to the reader.
    Reference and Attribution Management: For each image sourced, automatically generate and include appropriate references and attributions, 
    ensuring compliance with copyright laws and ethical standards.
    """)

In [33]:
ads_writter_node = functools.partial(
    agent_node, agent=ads_writter_agent, name="ads_writter")

In [34]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str


In [35]:
# Create workflow or graph
workflow = StateGraph(AgentState)

# adding nodes
workflow.add_node(node="supervisor", action=supervisor_chain)
workflow.add_node(node="news_correspondent", action=news_correspondent_node)
workflow.add_node(node="news_editor", action=news_editor_node)
workflow.add_node(node="ads_writter", action=ads_writter_node)


In [36]:
# define edgs
for member in members:
    workflow.add_edge(start_key=member, end_key="supervisor")

In [37]:
conditional_map = {k: k for k in members}
conditional_map['FINISH'] = END

In [38]:
# if task is FINISHED, supervisor won't send task to agent, else,
# the supervisor will keep on sending task to agent untill done, this is
# what the conditional edge does.
workflow.add_conditional_edges(
    "supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

In [ ]:
print(workflow.branches)
print(workflow.edges)
print(workflow.nodes)
print(workflow.channels)

In [40]:
graph = workflow.compile()

In [ ]:
# 1. Stream
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="""Write me a report on spaceX. After the research on spaceX,
                              pass the findings to the news editor to generate the final publication.
                              Once done, pass it to the ads writter to write the ads on the subject."""
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150}
):
    if not "__end__" in s:
        print(s, end="\n\n-----------------\n\n")


In [ ]:
# 2. No Streaming
final_respone = graph.invoke({
    "messages": [HumanMessage(content="""Write me a report on spaceX. After the research on spaceX,
                              pass the findings to the news editor to generate the final publication.
                              Once done, pass it to the ads writter to write the ads on the subject.""")]
}, {"recursion_limit": 150})

# print(final_respone["messages"][1].content)


In [ ]:
def run_graph(input_message):
    response = graph.invoke({
        "messages": [HumanMessage(content=input_message)]
    }, {"recursion_limit": 150})
    return response['messages'][1].content


In [ ]:
inputs = gr.components.Textbox(lines=2, placeholder="Enter your query here...")
outputs = gr.components.Markdown()


In [ ]:
demo = gr.Interface(
    fn=run_graph,
    inputs=inputs,
    outputs=outputs
)

demo.launch()